# Get Clean Merged and Seperate PL Data

In [41]:
import os, json
from pl_module import get_pl_keywords

In [42]:
Raw_Data_dir = "./Raw_Data/"
Data_dir = "./Data/"

all_data_path = "./data.txt"

In [43]:
# ['jobID', 'jobTitle', 'jobEmployer', 'jobLocation', 'jobPostTime', 'skills', 'employmentType', 'baseSalary', 'jobDescription', 'url']

w = open(all_data_path, "w", encoding='utf8')
raw_files = os.listdir(Raw_Data_dir)
for file in raw_files:
    if file.startswith("."): continue # skip .ipynb_checkpoints/ file

    ws = open(Data_dir + file + ".txt", "w", encoding="utf8")
    content = json.loads(open(Raw_Data_dir + file).read())
    for field in content:
        print(field, len(content[field]))
        
        for each in content[field]:
            keywords = get_pl_keywords(each["jobDescription"])
                   
            ws.write(' '.join(keywords) + "\n")
            w.write(' '.join(keywords) + "\n")

    ws.close()
w.close()

security engineer 776
android engineer 433
Backend 394
Frontend 831


# Get Vector for Used Words by Subprocess (NOT WORKING)

In [ ]:
# import subprocess
# subprocess.run(["/opt/fastText/fasttext", "print-word-vectors", "../wiki.en.bin", "<", "data.txt", ">", "used.words.vec"])
# # Not working

/opt/fastText/fasttext print-word-vectors wiki.en.bin < Steeve_model/data.txt > used.words.vec

# Read Model

In [44]:
import numpy as np

In [45]:
def read_model(path):
    fs = open(path, "r", encoding='utf8')
    
    model = {}
    for line in fs:
        each = line.strip().split(" ")
        model[each[0]] = np.array(each[1:]).astype(np.float32)
    return model

model_path = '../used.words.vec'
model = read_model(model_path)

# Convert to Vector

In [46]:
def para_2_vector(string):
    tokens = string.strip().split(" ")
    
    para_vec = np.zeros(300)
    for t in tokens:
        if t == '': continue
            
        para_vec += model[t]
    return para_vec

In [47]:
X = []
y = []
labels = {}

files = os.listdir(Data_dir)
for i, file in enumerate(files):
    if file.startswith("."): continue
    
    labels[i] = file    
    for line in open(Data_dir + file, "r", encoding='utf8'):
        vec = para_2_vector(line)
        X.append({"vec": vec, "src": line})
        y.append(i)

# Seperate to Training and Testing Data

In [48]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2)

print("Training data length: {}, Test data length: {}".format(len(train_X), len(test_X)))

Training data length: 1947, Test data length: 487


# Train and Predict

In [49]:
from sklearn import svm, metrics

# 建立 SVC 模型
svc = svm.SVC()
temp_X = list(map(lambda el: el["vec"], train_X))
svc_fit = svc.fit(temp_X, train_y)

# 預測
temp_X = list(map(lambda el: el["vec"], test_X))
_y = svc.predict(temp_X)

# 準確
acc = metrics.accuracy_score(test_y, _y)
print(acc)

0.845995893224


In [50]:
def get_name(string):
    return string.split(".")[0]
    
for i, val in enumerate(_y):
    if val == test_y[i]: continue

    print("Answer: {}, Predict: {}\n{}".format(get_name(labels[test_y[i]]), get_name(labels[val]), test_X[i]["src"]))

Answer: Security, Predict: Frontend
mobile mobile security android ios android ios mobile security this mobile testing mobile security mobile database mobile security mobile security testing mobile security security security this position security mobile security Python php javascript android mobile security click HERE position this position click this email search

Answer: andriod, Predict: Frontend
Java kernel android kernel android frameworks android Java interface linux android android android android android kernel Java android this position android Java Java android kernel kernel android frameworks package android package frameworks android Java interface jni linux rest android click HERE position this position click this email search

Answer: andriod, Predict: Frontend
android kotlin Java cloud 3d maps maps include android android process android package Java android android c android frameworks time android this position android Java background spring frameworks kotlin design p